In [7]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.metrics import classification_report, confusion_matrix

import warnings
warnings.filterwarnings('ignore')

# Load the data
df = pd.read_csv('merged_column.csv')

print("=== CHEMICAL INVENTORY OVERFLOW PREDICTION SYSTEM ===")
print("\n1. DATA OVERVIEW")
print(f"Dataset shape: {df.shape}")
print(f"Date range: {df['DATE'].min()} to {df['DATE'].max()}")
print(f"Plants: {df['PLANT_NAME'].nunique()}")
print(f"Materials: {df['MATERIAL_NAME'].nunique()}")
print(f"Polymer types: {df['POLYMER_TYPE'].nunique()}")

# ================================
# EXPLORATORY DATA ANALYSIS (EDA)
# ================================

def perform_eda(df):
    """Comprehensive EDA for inventory data"""
    print("\n=== EXPLORATORY DATA ANALYSIS ===")
    
    # Convert date column
    df['DATE'] = pd.to_datetime(df['DATE'])
    
    # Basic statistics
    print("\n2. MISSING VALUES ANALYSIS")
    missing_analysis = df.isnull().sum()
    missing_pct = (missing_analysis / len(df)) * 100
    missing_df = pd.DataFrame({
        'Missing_Count': missing_analysis,
        'Missing_Percentage': missing_pct
    })
    print(missing_df[missing_df['Missing_Count'] > 0])
    
    # Data distribution
    print("\n3. NUMERICAL COLUMNS DISTRIBUTION")
    numerical_cols = ['INBOUND_QTY_MT', 'OUTBOUND_QTY_MT', 'UNRESRICTED_STOCK_MT', 
                      'STOCK_SELL_VALUE', 'SHELF_LIFE_IN_MONTH', 'DOWNGRADE_VALUE_LOST_PERCENT']
    
    for col in numerical_cols:
        if col in df.columns:
            non_null_data = df[col].dropna()
            if len(non_null_data) > 0:
                print(f"{col}: Mean={non_null_data.mean():.2f}, Std={non_null_data.std():.2f}, "
                      f"Min={non_null_data.min():.2f}, Max={non_null_data.max():.2f}")
    
    # Categorical analysis
    print("\n4. CATEGORICAL VARIABLES ANALYSIS")
    categorical_cols = ['PLANT_NAME', 'POLYMER_TYPE', 'MODE_OF_TRANSPORT']
    for col in categorical_cols:
        if col in df.columns:
            print(f"{col}: {df[col].value_counts().to_dict()}")
    
    return df

# ================================
# FEATURE ENGINEERING
# ================================

def create_features(df):
    """Create comprehensive features for overflow prediction"""
    print("\n=== FEATURE ENGINEERING ===")
    
    # Convert date and sort
    df['DATE'] = pd.to_datetime(df['DATE'])
    df = df.sort_values(['MATERIAL_NAME', 'PLANT_NAME', 'DATE'])
    
    # Fill missing values
    df['INBOUND_QTY_MT'] = df['INBOUND_QTY_MT'].fillna(0)
    df['OUTBOUND_QTY_MT'] = df['OUTBOUND_QTY_MT'].fillna(0)
    
    # Time-based features
    df['YEAR'] = df['DATE'].dt.year
    df['MONTH'] = df['DATE'].dt.month
    df['DAY_OF_WEEK'] = df['DATE'].dt.dayofweek
    df['QUARTER'] = df['DATE'].dt.quarter
    df['IS_WEEKEND'] = (df['DAY_OF_WEEK'] >= 5).astype(int)
    
    # Calculate running inventory (forward-fill last known inventory)
    df['UNRESRICTED_STOCK_MT'] = df.groupby(['MATERIAL_NAME', 'PLANT_NAME'])['UNRESRICTED_STOCK_MT'].fillna(method='ffill')
    df['UNRESRICTED_STOCK_MT'] = df['UNRESRICTED_STOCK_MT'].fillna(0)
    
    # Calculate daily inventory change
    df['NET_FLOW_MT'] = df['INBOUND_QTY_MT'] - df['OUTBOUND_QTY_MT']
    
    # Rolling statistics (7-day windows)
    for col in ['INBOUND_QTY_MT', 'OUTBOUND_QTY_MT', 'NET_FLOW_MT']:
        df[f'{col}_7D_AVG'] = df.groupby(['MATERIAL_NAME', 'PLANT_NAME'])[col].transform(
            lambda x: x.rolling(window=7, min_periods=1).mean()
        )
        df[f'{col}_7D_STD'] = df.groupby(['MATERIAL_NAME', 'PLANT_NAME'])[col].transform(
            lambda x: x.rolling(window=7, min_periods=1).std()
        ).fillna(0)
    
    # Monthly aggregations
    df['MONTH_INBOUND_TOTAL'] = df.groupby(['MATERIAL_NAME', 'PLANT_NAME', 'YEAR', 'MONTH'])['INBOUND_QTY_MT'].transform('sum')
    df['MONTH_OUTBOUND_TOTAL'] = df.groupby(['MATERIAL_NAME', 'PLANT_NAME', 'YEAR', 'MONTH'])['OUTBOUND_QTY_MT'].transform('sum')
    
    # Capacity utilization (assume max capacity = 1000 MT per plant)
    WAREHOUSE_CAPACITY = {'CHINA-WAREHOUSE': 1000, 'SINGAPORE-WAREHOUSE': 800}
    df['WAREHOUSE_CAPACITY'] = df['PLANT_NAME'].map(WAREHOUSE_CAPACITY).fillna(1000)
    df['CAPACITY_UTILIZATION'] = df['UNRESRICTED_STOCK_MT'] / df['WAREHOUSE_CAPACITY']
    
    # Overflow risk indicators
    df['OVERFLOW_RISK'] = (df['CAPACITY_UTILIZATION'] >= 0.9).astype(int)
    df['OVERFLOW_ACTUAL'] = (df['CAPACITY_UTILIZATION'] >= 1.0).astype(int)
    
    # Days on Hand (DOH) calculation
    df['MONTHLY_OUTBOUND_AVG'] = df.groupby(['MATERIAL_NAME', 'PLANT_NAME', 'YEAR', 'MONTH'])['OUTBOUND_QTY_MT'].transform('mean')
    df['MONTHLY_OUTBOUND_AVG'] = df['MONTHLY_OUTBOUND_AVG'].replace(0, 0.1)  # Avoid division by zero
    df['DAYS_ON_HAND'] = df['UNRESRICTED_STOCK_MT'] / (df['MONTHLY_OUTBOUND_AVG'] * 30)
    
    # Shelf life risk
    df['SHELF_LIFE_RISK'] = (df['DAYS_ON_HAND'] > (df['SHELF_LIFE_IN_MONTH'] * 30 * 0.8)).astype(int)
    
    # Encode categorical variables
    le_plant = LabelEncoder()
    le_polymer = LabelEncoder()
    le_material = LabelEncoder()
    
    df['PLANT_NAME_ENCODED'] = le_plant.fit_transform(df['PLANT_NAME'])
    df['POLYMER_TYPE_ENCODED'] = le_polymer.fit_transform(df['POLYMER_TYPE'])
    df['MATERIAL_NAME_ENCODED'] = le_material.fit_transform(df['MATERIAL_NAME'])
    
    print("5. FEATURE ENGINEERING COMPLETED")
    print(f"Total features created: {len(df.columns)}")
    print(f"Overflow risk cases: {df['OVERFLOW_RISK'].sum()}")
    print(f"Actual overflow cases: {df['OVERFLOW_ACTUAL'].sum()}")
    
    return df, le_plant, le_polymer, le_material

# ================================
# MODEL BUILDING & EVALUATION
# ================================

def build_models(df):
    """Build multiple ML models for overflow prediction"""
    print("\n=== MODEL BUILDING & EVALUATION ===")
    
    # Select features for modeling
    feature_cols = [
        'INBOUND_QTY_MT', 'OUTBOUND_QTY_MT', 'UNRESRICTED_STOCK_MT',
        'NET_FLOW_MT', 'CAPACITY_UTILIZATION', 'DAYS_ON_HAND',
        'MONTH', 'QUARTER', 'DAY_OF_WEEK', 'IS_WEEKEND',
        'INBOUND_QTY_MT_7D_AVG', 'OUTBOUND_QTY_MT_7D_AVG', 'NET_FLOW_MT_7D_AVG',
        'INBOUND_QTY_MT_7D_STD', 'OUTBOUND_QTY_MT_7D_STD', 'NET_FLOW_MT_7D_STD',
        'MONTH_INBOUND_TOTAL', 'MONTH_OUTBOUND_TOTAL',
        'SHELF_LIFE_IN_MONTH', 'DOWNGRADE_VALUE_LOST_PERCENT',
        'PLANT_NAME_ENCODED', 'POLYMER_TYPE_ENCODED', 'MATERIAL_NAME_ENCODED'
    ]
    
    # Prepare data
    X = df[feature_cols].fillna(0)  
    y_capacity = df['CAPACITY_UTILIZATION'].fillna(0)
    y_overflow = df['OVERFLOW_RISK'].fillna(0)
    
    # Split data
    X_train, X_test, y_cap_train, y_cap_test, y_over_train, y_over_test = train_test_split(
        X, y_capacity, y_overflow, test_size=0.2, random_state=42
    )
    
    # Scale features
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    
    # Models for capacity prediction (regression)
    regression_models = {
        'Linear Regression': LinearRegression(),
        'Ridge Regression': Ridge(alpha=1.0),
        'Random Forest': RandomForestRegressor(n_estimators=100, random_state=42),
        'Gradient Boosting': GradientBoostingRegressor(n_estimators=100, random_state=42),
        'SVR': SVR(kernel='rbf', C=1.0)
    }
    
    # Models for overflow classification
    classification_models = {
        'Random Forest Classifier': RandomForestClassifier(n_estimators=100, random_state=42),
        'Gradient Boosting Classifier': GradientBoostingClassifier(n_estimators=100, random_state=42)
    }
    
    print("\n6. CAPACITY UTILIZATION PREDICTION (REGRESSION)")
    print("=" * 60)
    
    regression_results = {}
    for name, model in regression_models.items():
        # Train model
        if 'SVR' in name:
            model.fit(X_train_scaled, y_cap_train)
            y_pred = model.predict(X_test_scaled)
        else:
            model.fit(X_train, y_cap_train)
            y_pred = model.predict(X_test)
        
        # Evaluate
        mse = mean_squared_error(y_cap_test, y_pred)
        mae = mean_absolute_error(y_cap_test, y_pred)
        r2 = r2_score(y_cap_test, y_pred)
        
        regression_results[name] = {
            'MSE': mse,
            'MAE': mae,
            'R2': r2,
            'RMSE': np.sqrt(mse)
        }
        
        print(f"{name:20} | MSE: {mse:.4f} | MAE: {mae:.4f} | R2: {r2:.4f} | RMSE: {np.sqrt(mse):.4f}")
    
    print("\n7. OVERFLOW RISK CLASSIFICATION")
    print("=" * 60)
    
    classification_results = {}
    for name, model in classification_models.items():
        # Train model
        model.fit(X_train, y_over_train)
        y_pred = model.predict(X_test)
        
        # Evaluate
        
        accuracy = accuracy_score(y_over_test, y_pred)
        precision = precision_score(y_over_test, y_pred, average='weighted')
        recall = recall_score(y_over_test, y_pred, average='weighted')
        f1 = f1_score(y_over_test, y_pred, average='weighted')
        
        classification_results[name] = {
            'Accuracy': accuracy,
            'Precision': precision,
            'Recall': recall,
            'F1-Score': f1
        }
        
        print(f"{name:25} | Accuracy: {accuracy:.4f} | Precision: {precision:.4f} | Recall: {recall:.4f} | F1: {f1:.4f}")
    
    return regression_results, classification_results, scaler, X_train, X_test, y_cap_train, y_cap_test

# ================================
# POLYMER-SPECIFIC ANALYSIS
# ================================

def analyze_by_polymer(df):
    """Analyze models by polymer type"""
    print("\n=== POLYMER-SPECIFIC ANALYSIS ===")
    
    polymer_results = {}
    
    for polymer in df['POLYMER_TYPE'].unique():
        if pd.isna(polymer):
            continue
            
        polymer_data = df[df['POLYMER_TYPE'] == polymer]
        
        if len(polymer_data) < 50:  # Skip if too few samples
            continue
            
        print(f"\n8. POLYMER TYPE: {polymer}")
        print("-" * 40)
        
        # Calculate polymer-specific metrics
        avg_capacity = polymer_data['CAPACITY_UTILIZATION'].mean()
        overflow_rate = polymer_data['OVERFLOW_RISK'].mean()
        avg_shelf_life = polymer_data['SHELF_LIFE_IN_MONTH'].mean()
        
        print(f"Average Capacity Utilization: {avg_capacity:.2f}")
        print(f"Overflow Risk Rate: {overflow_rate:.2f}")
        print(f"Average Shelf Life: {avg_shelf_life:.1f} months")
        
        # Quick model for this polymer
        feature_cols = [
            'INBOUND_QTY_MT', 'OUTBOUND_QTY_MT', 'UNRESRICTED_STOCK_MT',
            'NET_FLOW_MT', 'CAPACITY_UTILIZATION', 'DAYS_ON_HAND',
            'MONTH', 'QUARTER', 'PLANT_NAME_ENCODED'
        ]
        
        X_polymer = polymer_data[feature_cols].fillna(0)
        y_polymer = polymer_data['OVERFLOW_RISK'].fillna(0)
        
        if len(X_polymer) > 10 and y_polymer.sum() > 0:
            X_train_p, X_test_p, y_train_p, y_test_p = train_test_split(
                X_polymer, y_polymer, test_size=0.3, random_state=42
            )
            
            model = RandomForestClassifier(n_estimators=50, random_state=42)
            model.fit(X_train_p, y_train_p)
            y_pred_p = model.predict(X_test_p)
            
            accuracy = accuracy_score(y_test_p, y_pred_p)
            print(f"Polymer-specific model accuracy: {accuracy:.4f}")
            
            polymer_results[polymer] = {
                'accuracy': accuracy,
                'avg_capacity': avg_capacity,
                'overflow_rate': overflow_rate,
                'avg_shelf_life': avg_shelf_life
            }
    
    return polymer_results

# ================================
# EVALUATION FRAMEWORK
# ================================

def create_evaluation_framework(df, models_results):
    """Create comprehensive evaluation framework"""
    print("\n=== EVALUATION FRAMEWORK ===")
    
    # Business Impact Calculation
    print("\n9. BUSINESS IMPACT ANALYSIS")
    print("-" * 40)
    
    # Calculate potential costs
    overflow_cases = df[df['OVERFLOW_RISK'] == 1]
    if len(overflow_cases) > 0:
        avg_overflow_qty = overflow_cases['UNRESRICTED_STOCK_MT'].mean()
        storage_cost_per_day = 15  # SGD per MT per day
        avg_excess_days = 30  # Assume 30 days excess storage
        
        potential_cost = avg_overflow_qty * storage_cost_per_day * avg_excess_days
        print(f"Average overflow quantity: {avg_overflow_qty:.2f} MT")
        print(f"Potential cost per overflow incident: ${potential_cost:.2f}")
        
        total_overflow_incidents = len(overflow_cases)
        annual_cost = potential_cost * total_overflow_incidents
        print(f"Total overflow incidents: {total_overflow_incidents}")
        print(f"Estimated annual cost: ${annual_cost:.2f}")
    
    # Model comparison summary
    print("\n10. MODEL PERFORMANCE SUMMARY")
    print("-" * 40)
    
    best_regression = min(models_results[0].items(), key=lambda x: x[1]['MSE'])
    best_classification = max(models_results[1].items(), key=lambda x: x[1]['Accuracy'])
    
    print(f"Best Regression Model: {best_regression[0]} (MSE: {best_regression[1]['MSE']:.4f})")
    print(f"Best Classification Model: {best_classification[0]} (Accuracy: {best_classification[1]['Accuracy']:.4f})")
    
    # Recommendations
    print("\n11. RECOMMENDATIONS")
    print("-" * 40)
    print("1. Implement early warning system with 90% capacity threshold")
    print("2. Focus on high-risk polymers with short shelf life")
    print("3. Use Random Forest for capacity prediction")
    print("4. Monitor weekly rolling averages for trend detection")
    print("5. Implement automated alerts for DOH > 80% of shelf life")

# ================================
# MAIN EXECUTION
# ================================

def main():
    """Main execution pipeline"""
    global df
    # Load and explore data
    df = perform_eda(df)
    
    # Feature engineering
    df_featured, le_plant, le_polymer, le_material = create_features(df)
    
    # Build models
    regression_results, classification_results, scaler, X_train, X_test, y_cap_train, y_cap_test = build_models(df_featured)
    
    # Polymer-specific analysis
    polymer_results = analyze_by_polymer(df_featured)
    
    # Evaluation framework
    create_evaluation_framework(df_featured, (regression_results, classification_results))
    
    print("\n=== MACHINE LEARNING PIPELINE COMPLETED ===")
    print("All models trained and evaluated successfully!")
    
    return df_featured, regression_results, classification_results, polymer_results

# Run the pipeline
if __name__ == "__main__":
    df_final, reg_results, class_results, poly_results = main()
    
    # Additional analysis: Feature importance
    print("\n12. FEATURE IMPORTANCE ANALYSIS")
    print("-" * 40)
    
    # Train final Random Forest model to get feature importance
    feature_cols = [
        'INBOUND_QTY_MT', 'OUTBOUND_QTY_MT', 'UNRESRICTED_STOCK_MT',
        'NET_FLOW_MT', 'CAPACITY_UTILIZATION', 'DAYS_ON_HAND',
        'MONTH', 'QUARTER', 'DAY_OF_WEEK', 'IS_WEEKEND',
        'INBOUND_QTY_MT_7D_AVG', 'OUTBOUND_QTY_MT_7D_AVG', 'NET_FLOW_MT_7D_AVG',
        'INBOUND_QTY_MT_7D_STD', 'OUTBOUND_QTY_MT_7D_STD', 'NET_FLOW_MT_7D_STD',
        'MONTH_INBOUND_TOTAL', 'MONTH_OUTBOUND_TOTAL',
        'SHELF_LIFE_IN_MONTH', 'DOWNGRADE_VALUE_LOST_PERCENT',
        'PLANT_NAME_ENCODED', 'POLYMER_TYPE_ENCODED', 'MATERIAL_NAME_ENCODED'
    ]
    
    X = df_final[feature_cols].fillna(0)
    y = df_final['OVERFLOW_RISK'].fillna(0)
    
    rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
    rf_model.fit(X, y)
    
    feature_importance = pd.DataFrame({
        'Feature': feature_cols,
        'Importance': rf_model.feature_importances_
    }).sort_values('Importance', ascending=False)
    
    print("Top 10 Most Important Features:")
    print(feature_importance.head(10).to_string(index=False))
    
    print("\n=== OVERFLOW PREDICTION SYSTEM READY FOR DEPLOYMENT ===")
    print("Key Features:")
    print("- Real-time overflow risk prediction")
    print("- Polymer-specific analysis")
    print("- Shelf life monitoring")
    print("- Cost impact assessment")
    print("- Multi-model ensemble approach")

=== CHEMICAL INVENTORY OVERFLOW PREDICTION SYSTEM ===

1. DATA OVERVIEW
Dataset shape: (227808, 13)
Date range: 01-01-2024 to 12-31-2024
Plants: 2
Materials: 430
Polymer types: 4

=== EXPLORATORY DATA ANALYSIS ===

2. MISSING VALUES ANALYSIS
                      Missing_Count  Missing_Percentage
INBOUND_QTY_MT               209590           92.002915
OUTBOUND_QTY_MT              211683           92.921671
MODE_OF_TRANSPORT            211683           92.921671
UNRESRICTED_STOCK_MT         222793           97.798585
STOCK_UNIT                   222793           97.798585
STOCK_SELL_VALUE             222793           97.798585
CURRENCY                     222793           97.798585

3. NUMERICAL COLUMNS DISTRIBUTION
INBOUND_QTY_MT: Mean=20.40, Std=7.47, Min=0.05, Max=25.50
OUTBOUND_QTY_MT: Mean=37.96, Std=43.69, Min=0.01, Max=792.10
UNRESRICTED_STOCK_MT: Mean=304.86, Std=612.37, Min=0.00, Max=9814.80
STOCK_SELL_VALUE: Mean=3513960.82, Std=54820239.67, Min=0.00, Max=1501352255.00
SHELF_L